# CBOW

In [1]:
import pandas as pd 

artigo_treino = pd.read_csv("../Aula 5 /treino.csv")
artigo_teste = pd.read_csv("../Aula 5 /teste.csv")

In [2]:
# pip install -U pip setuptools wheel
# pip install -U spacy
# python -m spacy download pt_core_news_sm

In [3]:
import spacy

nlp = spacy.load("pt_core_news_sm")

In [4]:
texto = "Adoro a cidade de Caldas novas"

doc = nlp(texto)

type(doc)

spacy.tokens.doc.Doc

In [5]:
doc.ents

(Adoro, Caldas novas)

In [6]:
doc[2].is_stop

False

In [7]:
doc[1].is_alpha # é alphanumerico

True

In [8]:
doc[1].is_stop #Verifica se é uma stopWord

True

In [9]:

def tratar_textos(doc): #tirar_stopwords e numeros 
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)
    if len(tokens_validos) > 2: #word2vec precisa no minimo uma frase com 3 palavras para o contexto
        return " ".join(tokens_validos)

In [10]:
texto = "Adoro 255255 cidade de Caldas Novas!"
doc = nlp(texto)
tratar_textos(doc)

'Adoro cidade Caldas'

In [ ]:
textos_para_tratamento = (titulos.lower() for titulos in artigo_treino.title)

textos_tratados = [tratar_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                         batch_size=100,
                                                         n_process= -1)]

In [ ]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.head()

In [ ]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0, # 0 para cbow e 1 skip gram 
               window= 2, # quantas palavras ele vai considerar uma antes e uma depois
               vector_size=300, # tamanho do vetor  
               min_count= 5, # eliminar as palavras  não recorrentes
               alpha= 0.3, # taxa de aprendizgem
               min_alpha= 0.007 # minimo de convergencia 
               )

In [ ]:
print(len(titulos_tratados))

titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

In [ ]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [ ]:
w2v_modelo.build_vocab(lista_lista_tokens)

In [ ]:
dir(w2v_modelo)

In [ ]:
w2v_modelo.corpus_count

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

# iniciando a chamada callback
class callback(CallbackAny2Vec):
  def __init__(self):
    self.epoch = 0

  def on_epoch_end(self, model):
    loss = model.get_latest_training_loss()
    if self.epoch == 0:
        print('Loss após a época {}: {}'.format(self.epoch, loss))
    else:
        print('Loss após a época {}: {}'.format(self.epoch, loss- self.loss_previous_step))
    self.epoch += 1
    self.loss_previous_step = loss

In [ ]:
w2v_modelo.train(lista_lista_tokens,
                 total_examples= w2v_modelo.corpus_count,
                 epochs= 30,
                 compute_loss= True,
                 callbacks=[callback()])

In [ ]:
w2v_modelo.wv.most_similar("google")

# Skip Gram


In [ ]:
from gensim.models import Word2Vec

w2v_modelo_sg = Word2Vec(sg = 1, # 0 para cbow e 1 skip gram 
               window= 5, # quantas palavras ele vai considerar uma antes e uma depois
               vector_size=300, # tamanho do vetor  
               min_count= 5, # eliminar as palavras  não recorrentes
               alpha= 0.3, # taxa de aprendizgem
               min_alpha= 0.007 # minimo de convergencia 
               )

w2v_modelo_sg.build_vocab(lista_lista_tokens)



In [ ]:
w2v_modelo_sg.train(lista_lista_tokens,
                 total_examples= w2v_modelo_sg.corpus_count,
                 epochs= 30,
                 compute_loss= True,
                 callbacks=[callback()])

In [ ]:
w2v_modelo_sg.wv.most_similar("google")

## Exportando modelos

In [ ]:
w2v_modelo.wv.save_word2vec_format("./meu_modelo_cbow.txt", binary= False)

In [ ]:
w2v_modelo_sg.wv.save_word2vec_format("./meu_modelo_skip_gram.txt", binary= False)

In [ ]:
from gensim.models import KeyedVectors

meu_modelo_cbow = KeyedVectors.load_word2vec_format("meu_modelo_cbow.txt")
meu_modelo_sg = KeyedVectors.load_word2vec_format("meu_modelo_skip_gram.txt")


In [ ]:
meu_modelo_cbow.most_similar("ana")

# Classificador

In [ ]:
# def tokenizador(texto):
#   tokens_validos = []
#   doc = nlp(texto)
#   for token in doc:
#     e_valido = not token.is_stop and token.is_alpha
#     if e_valido:
#       tokens_validos.append(token.text.lower())

#   return tokens_validos

#COMENTEI ESSE TOKENIZADOR POR QUE ELE NÃO FUNCIONA BEM COM SPACY QUE USA PROCESSAMENTO EM LOTES

# import numpy as np

# def combinacao_de_vetores_por_soma(palavras_numeros,modelo):
#   vetor_resultante = np.zeros(300)
#   for pn in palavras_numeros:
#     try: 
#       vetor_resultante =+ modelo.get_vector(pn)
#     except KeyError:
#       pass
#   return vetor_resultante
# def matriz_vetores(textos,modelo):
#     x = len(textos)
#     y = 300
#     matriz = np.zeros((x,y))

#     for i in range(x):
#         palavras = tokenizador(textos.iloc[i])
#         matriz[i]= combinacao_de_vetores_por_soma(palavras,modelo)
#     return matriz
# matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title,meu_modelo)
# matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title,meu_modelo)

# COMENTEI O RESTO DO CÓDIGO PQ += ESTA ERRADO NA FUNÇÃO  vetor_resultante =+ modelo.get_vector(pn)
#  E A LOGICA NÃO IA FUNCIONAR FAZENDO O TOKENIZADOR DIFERENTE ENTÃO PRECISAVA DE MODIFICAÇÃO
# O USO DO SPACY DEVERIA TER SIDO COMO NA FUNÇÃO NO COMEÇO DO NOTEBOOK TRATAR TEXTO


import spacy

nlp = spacy.load(
    "pt_core_news_sm",
    disable=["parser", "ner", "tagger", "textcat"]
)

def tokenizador(doc):
    return [
        token.text.lower()
        for token in doc
        if token.is_alpha and not token.is_stop
    ]

import numpy as np

def combinacao_de_vetores_por_soma(palavras, modelo):
    vetor_resultante = np.zeros(300)
    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)
        except KeyError:
            pass
    return vetor_resultante


def matriz_vetores(docs, modelo):
    x = len(docs)
    y = 300
    matriz = np.zeros((x, y))

    for i, doc in enumerate(docs):
        palavras = tokenizador(doc)
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz




In [ ]:
docs_treino = list(nlp.pipe(artigo_treino.title))
docs_teste  = list(nlp.pipe(artigo_teste.title))

matriz_vetores_treino_cbow = matriz_vetores(docs_treino, meu_modelo_cbow)
matriz_vetores_teste_cbow  = matriz_vetores(docs_teste, meu_modelo_cbow)
matriz_vetores_treino_sg = matriz_vetores(docs_treino, meu_modelo_sg)
matriz_vetores_teste_sg  = matriz_vetores(docs_teste, meu_modelo_sg)

# DA FORMA COMENTADA DEMOROU 20 MINUTOS PARA RODAR E AQUI 2MIN


In [ ]:
type(matriz_vetores_teste_cbow)

In [ ]:
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(x_treino,y_treino,x_teste, y_teste):
    LR = LogisticRegression(max_iter= 800)
    LR.fit(x_treino,y_treino)
    categorias = LR.predict(x_teste)
    CR = classification_report(y_teste,categorias)
    print(CR)



In [ ]:
LR_cbow = classificador(matriz_vetores_treino_cbow,
                        artigo_treino.category,
                        matriz_vetores_teste_cbow,
                        artigo_teste.category)

In [ ]:
LR_skip_gram = classificador(matriz_vetores_treino_sg,
                        artigo_treino.category,
                        matriz_vetores_teste_sg,
                        artigo_teste.category)